<a href="https://colab.research.google.com/github/raginisingh-del/Agentic-AI/blob/main/Fine_tuning_Lab_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

!pip install -q transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import math


In [4]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    logging_steps=100,
    report_to="none"
)


In [6]:
model_name = "distilgpt2"

# 1. Load the tokenizer (to turn text into numbers)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # Causal models need this for padding

# 2. Load the actual model
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [9]:
# 1. Load both a train and test split
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split={"train": "train[:1%]", "test": "test[:1%]"})

# 2. Tokenize both splits
tokenized_dataset = dataset.map(tokenize_func, batched=True, remove_columns=["text"])

# 3. Update your Trainer cell to include 'eval_dataset'
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"], # This is the missing piece!
    data_collator=data_collator,
)

Map:   0%|          | 0/367 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [10]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,4.092148


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=46, training_loss=4.456057341202445, metrics={'train_runtime': 447.5419, 'train_samples_per_second': 0.82, 'train_steps_per_second': 0.103, 'total_flos': 10688670203904.0, 'train_loss': 4.456057341202445, 'epoch': 1.0})

In [12]:
# Run this in a new cell
trainer.save_model("./my_finetuned_model")
tokenizer.save_pretrained("./my_finetuned_model")

print("Model and Tokenizer saved successfully to './my_finetuned_model'")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Model and Tokenizer saved successfully to './my_finetuned_model'


In [13]:
from transformers import pipeline

# 1. Setup the text generation pipeline using your saved model
generator = pipeline("text-generation", model="./my_finetuned_model", tokenizer="./my_finetuned_model")

# 2. Give the model a starting sentence
prompt = "The future of artificial intelligence is"

# 3. Generate text
output = generator(prompt, max_length=50, num_return_sequences=1)

# 4. Print the result
print("\n--- Model Output ---")
print(output[0]['generated_text'])

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

Passing `generation_config` together with generation-related arguments=({'num_return_sequences', 'max_length'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Model Output ---
The future of artificial intelligence is well known, from the original "Theorem" to the "Theorem" (in the first two chapters of the book), but even a number of major innovations have been made since the original "Theorem" was published in 1937.


















































































































































































































** Brief summary**:Step Explanation: "I used the Trainer API to fine-tune distilgpt2 on the wikitext dataset. I configured the model for 1 epoch with a batch size of 8."Results: "The final training loss was 4.456 and the validation loss was 4.092."Observation: "The validation loss is lower than the training loss, which suggests the model is generalizing well and not overfitting on this small sample. The perplexity (calculated as $e^{loss}$) shows the model's confidence in predicting the next token."

In [14]:
import math
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results['eval_loss'])
print(f"The model's Perplexity is: {perplexity:.2f}")

The model's Perplexity is: 59.87


# Fine-Tuning a Small Language Model (SLM) on Text Data

Model: distilgpt2  
Dataset: ag_news  
Platform: Google Colab  

This notebook demonstrates fine-tuning a small language model on a Hugging Face dataset.
